<a href="https://colab.research.google.com/github/jbrewer98/CS-1101-Debugging-Lecture/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import packages
import time
from copy import deepcopy
import json
import pymongo
import os
import datetime
import pandas as pd

In [2]:
#get the username and and password information - You need to run it one time at start.
# it will prompt you and ask for input
# test

from getpass import getpass

USER = getpass('Enter User Name here: ')
PASS = getpass('Enter Password here: ')
HOST = getpass('Enter HOST here (it should be the name of your AWS instace - you use it to connect to your instance) : ')
PORT = getpass('Enter PORT here - unless you changed it, it is the default mongodb port -27017: ')

Enter User Name here: ··········
Enter Password here: ··········
Enter HOST here (it should be the name of your AWS instace - you use it to connect to your instance) : ··········
Enter PORT here - unless you changed it, it is the default mongodb port -27017: ··········


In [0]:
#set up mongo connection
mongo_url = "mongodb://{}:{}@{}:{}/?authSource={}".format(USER,
                                                         PASS,
                                                         HOST,
                                                         PORT,
                                                         "lahman")

client = pymongo.MongoClient(mongo_url)

In [9]:
#get access to a specific database. this will work if you imported the data properly as instructed
db = client['lahman']
#now print all collection names
print("collection names: {}".format(db.list_collection_names()))
reading = db['Batting'].find_one()
#print one batting documents
reading["_id"] = str(reading["_id"])
print(reading)



collection names: ['Schools', 'FieldingOF', 'AwardsPlayers', 'Managers', 'Appearances', 'TeamsFranchises', 'AwardsManagers', 'SeriesPost', 'AwardsShareManagers', 'AwardsSharePlayers', 'SchoolsPlayers', 'Pitching', 'Batting', 'BattingPost', 'PitchingPost', 'ManagersHalf', 'HallOfFame', 'TeamsHalf', 'Fielding', 'Teams', 'FieldingPost', 'AllstarFull', 'Master', 'Salaries']
{'_id': '5e49faeb73e7093b20fb4375', 'playerID': 'aardsda01', 'yearID': 2004, 'stint': 1, 'teamID': 'SFN', 'lgID': 'NL', 'G': 11, 'G_batting': 11, 'AB': 0, 'R': 0, 'H': 0, '2B': 0, '3B': 0, 'HR': 0, 'RBI': 0, 'SB': 0, 'CS': 0, 'BB': 0, 'SO': 0, 'IBB': 0, 'HBP': 0, 'SH': 0, 'SF': 0, 'GIDP': 0, 'G_old': 11}


In [36]:
def query_1():
  # The number of all stars in allstarfull.

  query = {}
  result = db['AllstarFull'].count_documents(query)
  print(result)
  

query_1()

4834


In [59]:
def query_2():
  # The most home runs in a season by a single player (using the batting table).

  query = {}
  result = db['Batting'].find(query)
  data = []

  for x in result:
    if x["HR"] != "":
      data.append(x["HR"])

  data = list(map(int, data))
  print(max(data))
  
query_2()

73


In [78]:
def query_3():
  # The playerid of the player with the most home runs in a season.

  query = {}
  result = db['Batting'].find(query)
  data = []

  for x in result:
    if x["HR"] != "":
      data.append(x)

  df = pd.DataFrame(data = data)
  index = df["HR"].idxmax()
  print(df["playerID"][index])
  
query_3()

bondsba01


In [79]:
def query_4():
  # The number of leagues in the batting table.

  query = {}
  result = db['Batting'].find(query)
  data = [x for x in result]
  
  df = pd.DataFrame(data = data)
  print(df["lgID"].nunique())
  
query_4()

7


In [83]:
def query_5():
  # Barry Bond's average batting average (playerid = 'bondsba01') where batting average is hits / at-bats. 
  # Note you will nead to cast hits to get a decimal: cast(h as real)

  query = {"playerID": {"$eq": "bondsba01"}}
  result = db['Batting'].find(query)
  data = [x for x in result]
  
  df = pd.DataFrame(data = data)
  average = (df["H"]/df["AB"]).mean()
  print(average)
  
query_5()

0.2983057628600463


In [93]:
def query_6():
  # The teamid with the fewest hits in the year 2000 (ie., yearid = '2000').
  # Return both the teamid, and the number of hits.

  query = {"yearID": {"$eq": 2000}}
  result = db['Teams'].find(query)
  data = [x for x in result]
  
  
  df = pd.DataFrame(data = data)
  index = df["H"].idxmin()
  print("Team:",df["teamID"][index],"Hits:",df["H"][index])
  
query_6()

Team: MIL Hits: 1366


In [135]:
def query_7():
  # The teamid in the year 2000 (i.e., yearid = '2000') with the highest average batting average.
  # Return the teamid and the average. To prevent divsion by 0, limit at-bats > 0.

  query = {"yearID": {"$eq": 2000},"AB": {"$ne": ""}}
  result = db['Batting'].find(query)
  data = [x for x in result]
  
  df = pd.DataFrame(data = data)
  df['BAvg'] = df["H"]/df["AB"]

  teamAvg = df.groupby(['teamID'])['BAvg'].mean()
  index = teamAvg.idxmax()
  print(index,teamAvg[index])
  
  
query_7()

SEA 0.2807991584890046
